# test sqlite engine

In [ ]:
#| default_exp src.rgxlog_interpreter.tests.test_sqlite_engine

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
!pip install pytest

In [ ]:
#| export
import pytest
import pandas as pd
from spanner_workbench.src.rgxlog_interpreter.src.rgxlog.engine.datatypes.ast_node_types import *
from spanner_workbench.src.rgxlog_interpreter.src.rgxlog.engine.datatypes.primitive_types import Span, DataTypes, DataTypeMapping
from spanner_workbench.src.rgxlog_interpreter.src.rgxlog.engine.engine import SqliteEngine

In [ ]:
#| export
def test_add_fact() -> None:
    my_engine = SqliteEngine()
    expected_output = [
        (8, "hihi"),
        (10, "hello"),
        (15, "world")
    ]

    relation = RelationDeclaration("yoyo", [DataTypes.integer, DataTypes.string])
    my_engine.declare_relation_table(relation)

    facts = [
        AddFact("yoyo", [8, "hihi"], [DataTypes.integer, DataTypes.string]),
        AddFact("yoyo", [10, "hello"], [DataTypes.integer, DataTypes.string]),
        AddFact("yoyo", [15, "world"], [DataTypes.integer, DataTypes.string])
    ]

    for fact in facts:
        my_engine.add_fact(fact)

    result = my_engine.query(Query("yoyo", ["X", "Y"], [DataTypes.free_var_name, DataTypes.free_var_name]))

    assert expected_output == result


    relation1 = RelationDeclaration("numbers", [DataTypes.integer])
    relation2 = RelationDeclaration("words", [DataTypes.string])
    my_engine.declare_relation_table(relation1)
    my_engine.declare_relation_table(relation2)

    facts = [
        AddFact("numbers", [5], [DataTypes.integer]),
        AddFact("numbers", [10], [DataTypes.integer]),
        AddFact("words", ["apple"], [DataTypes.string]),
        AddFact("words", ["banana"], [DataTypes.string])
    ]

    for fact in facts:
        my_engine.add_fact(fact)

    result_numbers = my_engine.query(Query("numbers", ["X"], [DataTypes.free_var_name]))
    result_words = my_engine.query(Query("words", ["W"], [DataTypes.free_var_name]))

    expected_numbers_output = [(5,), (10,)]
    expected_words_output = [("apple",), ("banana",)]

    assert expected_numbers_output == result_numbers
    assert expected_words_output == result_words

    relation3 = RelationDeclaration("students", [DataTypes.string, DataTypes.integer])
    relation4 = RelationDeclaration("courses", [DataTypes.string])
    my_engine.declare_relation_table(relation3)
    my_engine.declare_relation_table(relation4)

    facts = [
        AddFact("students", ["Alice", 20], [DataTypes.string, DataTypes.integer]),
        AddFact("students", ["Bob", 22], [DataTypes.string, DataTypes.integer]),
        AddFact("students", ["Eve", 19], [DataTypes.string, DataTypes.integer]),
        AddFact("courses", ["Math"], [DataTypes.string]),
        AddFact("courses", ["Physics"], [DataTypes.string])
    ]

    for fact in facts:
        my_engine.add_fact(fact)

    query3 = Query("students", ["Name", "Age"], [DataTypes.free_var_name, DataTypes.free_var_name])
    query4 = Query("courses", ["Course"], [DataTypes.free_var_name])

    result_students = my_engine.query(query3)
    result_courses = my_engine.query(query4)

    expected_students_output = [("Alice", 20), ("Bob", 22), ("Eve", 19)]
    expected_courses_output = [("Math",), ("Physics",)]

    assert expected_students_output == result_students
    assert expected_courses_output == result_courses

In [ ]:
#| export
def test_remove_fact() -> None:
    my_engine = SqliteEngine()
    expected_output = [
        (10, "hello"),
        (15, "world")
    ]

    relation = RelationDeclaration("yoyo", [DataTypes.integer, DataTypes.string])
    my_engine.declare_relation_table(relation)

    facts = [
        AddFact("yoyo", [8, "hihi"], [DataTypes.integer, DataTypes.string]),
        AddFact("yoyo", [10, "hello"], [DataTypes.integer, DataTypes.string]),
        AddFact("yoyo", [15, "world"], [DataTypes.integer, DataTypes.string])
    ]

    for fact in facts:
        my_engine.add_fact(fact)
    
    my_engine.remove_fact(RemoveFact("yoyo", [8, "hihi"], [DataTypes.integer, DataTypes.string]))

    result = my_engine.query(Query("yoyo", ["X", "Y"], [DataTypes.free_var_name, DataTypes.free_var_name]))

    assert expected_output == result

    my_engine.remove_fact(RemoveFact("yoyo", [8, "hihi"], [DataTypes.integer, DataTypes.string]))

    result = my_engine.query(Query("yoyo", ["X", "Y"], [DataTypes.free_var_name, DataTypes.free_var_name]))

    assert expected_output == result

    

In [ ]:
#| export
def test_remove_all_facts_and_add_back() -> None:
    my_engine = SqliteEngine()
    expected_empty_output = []

    relation = RelationDeclaration("relation", [DataTypes.integer, DataTypes.string, DataTypes.integer])
    my_engine.declare_relation_table(relation)


    facts = [
        AddFact("relation", [8, "hihi", 3], [DataTypes.integer, DataTypes.string, DataTypes.integer]),
        AddFact("relation", [10, "hello", 2], [DataTypes.integer, DataTypes.string, DataTypes.integer]),
        AddFact("relation", [15, "world", 1], [DataTypes.integer, DataTypes.string, DataTypes.integer])
    ]

    for fact in facts:
        my_engine.add_fact(fact)

    my_engine.remove_fact(RemoveFact("relation", [8, "hihi", 3], [DataTypes.integer, DataTypes.string, DataTypes.integer]))
    my_engine.remove_fact(RemoveFact("relation", [10, "hello", 2], [DataTypes.integer, DataTypes.string, DataTypes.integer]))
    my_engine.remove_fact(RemoveFact("relation", [15, "world", 1], [DataTypes.integer, DataTypes.string, DataTypes.integer]))


    empty_result = my_engine.query(Query("relation", ["X", "Y", "Z"],
                                         [DataTypes.free_var_name, DataTypes.free_var_name, DataTypes.free_var_name]))

    assert expected_empty_output == empty_result

    for fact in facts:
        my_engine.add_fact(fact)

    expected_output = [
        (8, 'hihi', 3),
        (10, 'hello', 2),
        (15, 'world', 1)
    ]

    added_result = my_engine.query(Query("relation", ["X", "Y", "Z"],
                                        [DataTypes.free_var_name, DataTypes.free_var_name, DataTypes.free_var_name]))

    assert expected_output == added_result

In [ ]:
#| export
def test_query() -> None:
    my_engine = SqliteEngine()

    relation = RelationDeclaration("people", [DataTypes.integer, DataTypes.string])
    my_engine.declare_relation_table(relation)

    facts = [
        AddFact("people", [1, "Alice"], [DataTypes.integer, DataTypes.string]),
        AddFact("people", [2, "Bob"], [DataTypes.integer, DataTypes.string]),
        AddFact("people", [3, "Charlie"], [DataTypes.integer, DataTypes.string])
    ]

    for fact in facts:
        my_engine.add_fact(fact)

    query_result = my_engine.query(Query("people", ["ID", "Name"],
                                         [DataTypes.free_var_name, DataTypes.free_var_name]))

    expected_output = [
        (1, "Alice"),
        (2, "Bob"),
        (3, "Charlie")
    ]

    assert expected_output == query_result

    courses_relation = RelationDeclaration("courses", [DataTypes.string, DataTypes.integer])
    my_engine.declare_relation_table(courses_relation)

    facts = [
        AddFact("courses", ["Math", 101], [DataTypes.string, DataTypes.integer]),
        AddFact("courses", ["Science", 202], [DataTypes.string, DataTypes.integer]),
        AddFact("courses", ["History", 303], [DataTypes.string, DataTypes.integer])
    ]

    for fact in facts:
        my_engine.add_fact(fact)

    query_result = my_engine.query(Query("courses", ["Name", "Number"],
                                         [DataTypes.free_var_name, DataTypes.free_var_name]))

    expected_output = [
        ("Math", 101),
        ("Science", 202),
        ("History", 303)
    ]

    assert expected_output == query_result

In [ ]:
#| export
def test_query_sanity2() -> None:
    my_engine = SqliteEngine()

    relation1 = RelationDeclaration("important_years", [DataTypes.integer])
    relation2 = RelationDeclaration("courses", [DataTypes.string, DataTypes.integer])

    my_engine.declare_relation_table(relation1)
    my_engine.declare_relation_table(relation2)


    facts = [
        AddFact("important_years",[2000],[DataTypes.integer]),
        AddFact("important_years",[1900],[DataTypes.integer]),
        AddFact("important_years",[1800],[DataTypes.integer]),
        AddFact("important_years",[1700],[DataTypes.integer]),
        AddFact("important_years",[1600],[DataTypes.integer]),
        AddFact("courses", ["Math", 2000], [DataTypes.string, DataTypes.integer]),
        AddFact("courses", ["Science", 1900], [DataTypes.string, DataTypes.integer]),
        AddFact("courses", ["History", 1800], [DataTypes.string, DataTypes.integer])
    ]

    for fact in facts:
        my_engine.add_fact(fact)

    query_result = my_engine.query(Query("courses", ["Name", 1900],
                                         [DataTypes.free_var_name, DataTypes.integer]))
    
    assert query_result == [('Science',)]


In [ ]:
#| export
def test_query_sanity3() -> None:
    my_engine = SqliteEngine()

    relation1 = RelationDeclaration("people", [DataTypes.integer, DataTypes.string])
    relation2 = RelationDeclaration("courses", [DataTypes.string, DataTypes.string, DataTypes.integer])
    relation3 = RelationDeclaration("enrollments", [DataTypes.integer, DataTypes.string])

    my_engine.declare_relation_table(relation1)
    my_engine.declare_relation_table(relation2)
    my_engine.declare_relation_table(relation3)

    facts = [
        AddFact("people", [1, "Alice"], [DataTypes.integer, DataTypes.string]),
        AddFact("people", [2, "Bob"], [DataTypes.integer, DataTypes.string]),
        AddFact("people", [3, "Charlie"], [DataTypes.integer, DataTypes.string]),
        AddFact("courses", ["Math", "ProfA", 101], [DataTypes.string, DataTypes.string, DataTypes.integer]),
        AddFact("courses", ["Science", "ProfB", 202], [DataTypes.string, DataTypes.string, DataTypes.integer]),
        AddFact("courses", ["History", "ProfC", 303], [DataTypes.string, DataTypes.string, DataTypes.integer]),
        AddFact("enrollments", [1, "Math"], [DataTypes.integer, DataTypes.string]),
        AddFact("enrollments", [2, "Science"], [DataTypes.integer, DataTypes.string]),
        AddFact("enrollments", [3, "History"], [DataTypes.integer, DataTypes.string]),
    ]

    for fact in facts:
        my_engine.add_fact(fact)

    query_result = my_engine.query(Query("enrollments", ["X", "Y"],
                                         [DataTypes.free_var_name, DataTypes.free_var_name]))
    
    expected_output = [
        (1, "Math"),
        (2, "Science"),
        (3, "History")
    ]

    assert query_result == expected_output

In [ ]:
#| export
def test_query_complex() -> None:
    my_engine = SqliteEngine()
    relation = RelationDeclaration("common_relation", [DataTypes.integer, DataTypes.integer, DataTypes.string, DataTypes.integer, DataTypes.string])

    my_engine.declare_relation_table(relation)

    facts = [
        AddFact("common_relation", [1, 100, "Apple", 50, "Red"], [DataTypes.integer, DataTypes.integer, DataTypes.string, DataTypes.integer, DataTypes.string]),
        AddFact("common_relation", [2, 200, "Banana", 40, "Yellow"], [DataTypes.integer, DataTypes.integer, DataTypes.string, DataTypes.integer, DataTypes.string]),
        AddFact("common_relation", [3, 300, "Cherry", 30, "Red"], [DataTypes.integer, DataTypes.integer, DataTypes.string, DataTypes.integer, DataTypes.string]),
        AddFact("common_relation", [4, 400, "Date", 20, "Brown"], [DataTypes.integer, DataTypes.integer, DataTypes.string, DataTypes.integer, DataTypes.string]),
        AddFact("common_relation", [5, 500, "Elderberry", 10, "Purple"], [DataTypes.integer, DataTypes.integer, DataTypes.string, DataTypes.integer, DataTypes.string]),
        AddFact("common_relation", [6, 200, "Fig", 40, "Green"], [DataTypes.integer, DataTypes.integer, DataTypes.string, DataTypes.integer, DataTypes.string]),
        AddFact("common_relation", [7, 400, "Grape", 20, "Purple"], [DataTypes.integer, DataTypes.integer, DataTypes.string, DataTypes.integer, DataTypes.string]),
        AddFact("common_relation", [8, 100, "Apple", 50, "Red"], [DataTypes.integer, DataTypes.integer, DataTypes.string, DataTypes.integer, DataTypes.string]),
        AddFact("common_relation", [9, 500, "Elderberry", 10, "Purple"], [DataTypes.integer, DataTypes.integer, DataTypes.string, DataTypes.integer, DataTypes.string]),
        AddFact("common_relation", [10, 300, "Cherry", 30, "Red"], [DataTypes.integer, DataTypes.integer, DataTypes.string, DataTypes.integer, DataTypes.string])
    ]

    for fact in facts:
        my_engine.add_fact(fact)
    
    query_result = my_engine.query(Query("common_relation", ["X", 200, "Y", "Z", "W"], [DataTypes.free_var_name, DataTypes.integer, DataTypes.free_var_name, DataTypes.free_var_name, DataTypes.free_var_name]))


    expected_result = [
        (2, "Banana", 40, "Yellow"),
        (6, "Fig", 40, "Green")
    ]

    assert query_result == expected_result

    query_result = my_engine.query(Query("common_relation", ["X", 200, "Y", 40, "W"], [DataTypes.free_var_name, DataTypes.integer, DataTypes.free_var_name, DataTypes.integer, DataTypes.free_var_name]))

    expected_result = [
        (2, "Banana", "Yellow"),
        (6, "Fig", "Green")
    ]

    assert query_result == expected_result


    query_result = my_engine.query(Query("common_relation", ["X", "Y", "Y", 40, "W"], [DataTypes.free_var_name, DataTypes.free_var_name, DataTypes.free_var_name, DataTypes.integer, DataTypes.free_var_name]))

    expected_result = []

    assert query_result == expected_result


    query_result = my_engine.query(Query("common_relation", ["X", 100, "Apple", 50, "W"], [DataTypes.free_var_name, DataTypes.integer, DataTypes.string, DataTypes.integer, DataTypes.free_var_name]))

    expected_result = [
        (1, "Red"),
        (8, "Red")
    ]
    assert query_result == expected_result

    query_result = my_engine.query(Query("common_relation", ["X", 100, "Apple", 50, "Red"], [DataTypes.free_var_name, DataTypes.integer, DataTypes.string, DataTypes.integer, DataTypes.string]))

    expected_result = [
        (1,),
        (8,)
    ]
    assert query_result == expected_result


In [ ]:
#| export
def test_query_complex2() -> None:
    my_engine = SqliteEngine()

    relation = RelationDeclaration("wordy", [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string])

    my_engine.declare_relation_table(relation)

    facts = [
        AddFact("wordy", ["apple", "banana", "cherry", "date"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["banana", "cherry", "date", "elderberry"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["cherry", "date", "elderberry", "fig"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["date", "elderberry", "fig", "grape"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["elderberry", "fig", "grape", "honeydew"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["fig", "grape", "honeydew", "kiwi"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["grape", "honeydew", "kiwi", "lemon"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["honeydew", "kiwi", "lemon", "mango"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["pear", "kiwi", "lemon", "mango"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["kiwi", "lemon", "mango", "nectarine"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["lemon", "mango", "nectarine", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["mango", "nectarine", "pear", "quince"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["nectarine", "orange", "pear", "raspberry"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["orange", "pear", "quince", "raspberry"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["pear", "quince", "raspberry", "strawberry"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["quince", "raspberry", "strawberry", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["raspberry", "strawberry", "pear", "tangerine"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["strawberry", "tangerine", "pear", "uva"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["tangerine", "uva", "pear", "violet"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["uva", "violet", "pear", "watermelon"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["violet", "watermelon", "pear", "xigua"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["pear", "watermelon", "xigua", "yuzu"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["pear", "quince", "raspberry", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["mango", "pear", "quince", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string])
    ]

    for fact in facts:
        my_engine.add_fact(fact)

    query_result = my_engine.query(Query("wordy", ["X", "kiwi", "Y", "Z"], [DataTypes.free_var_name, DataTypes.string, DataTypes.free_var_name, DataTypes.free_var_name]))

    expected_result = [
        ("honeydew",  "lemon", "mango"),
        ("pear", "lemon", "mango")
    ]

    assert query_result == expected_result

    query_result = my_engine.query(Query("wordy", ["X", "Y", "X", "Z"], [DataTypes.free_var_name, DataTypes.free_var_name, DataTypes.free_var_name, DataTypes.free_var_name]))
    
    expected_result = [
        ("pear", "quince", "pear", "pear"),
    ]
    assert query_result == expected_result

    query_result = my_engine.query(Query("wordy", ["X", "Y", "Z", "X"], [DataTypes.free_var_name, DataTypes.free_var_name, DataTypes.free_var_name, DataTypes.free_var_name]))
    
    expected_result = [
    ("pear", "quince", "raspberry", "pear"),
    ("pear", "quince", "pear", "pear")

    ]
    assert query_result == expected_result

    query_result = my_engine.query(Query("wordy", ["X", "Y", "raspberry", "X"], [DataTypes.free_var_name, DataTypes.free_var_name, DataTypes.string, DataTypes.free_var_name]))
    

    expected_result = [
        ("pear", "quince", "pear")
    ]

    assert query_result == expected_result


    query_result = my_engine.query(Query("wordy", ["mango", "X", "quince", "X"], [DataTypes.string, DataTypes.free_var_name, DataTypes.string, DataTypes.free_var_name]))
    

    expected_result = [
        ("pear", "pear")
    ]
    
    assert query_result == expected_result

    query_result = my_engine.query(Query("wordy", ["pear", "quince", "X", "Y"], [DataTypes.string, DataTypes.string, DataTypes.free_var_name, DataTypes.free_var_name]))
    

    expected_result = [
        ("raspberry", "strawberry"),
        ("raspberry", "pear"),
        ("pear", "pear")
    ]
    
    assert query_result == expected_result

In [ ]:
#| export
def test_operator_select() -> None:
    my_engine = SqliteEngine()

    relation = RelationDeclaration("wordy", [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string])

    my_engine.declare_relation_table(relation)

    facts = [
        AddFact("wordy", ["apple", "banana", "cherry", "date"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["banana", "cherry", "date", "elderberry"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["cherry", "date", "elderberry", "fig"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["date", "elderberry", "fig", "grape"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["elderberry", "fig", "grape", "honeydew"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["fig", "grape", "honeydew", "kiwi"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["grape", "honeydew", "kiwi", "lemon"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["honeydew", "kiwi", "lemon", "mango"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["pear", "kiwi", "lemon", "mango"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["kiwi", "lemon", "mango", "nectarine"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["lemon", "mango", "nectarine", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["mango", "nectarine", "pear", "quince"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["nectarine", "orange", "pear", "raspberry"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["orange", "pear", "quince", "raspberry"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["pear", "quince", "raspberry", "strawberry"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["quince", "raspberry", "strawberry", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["raspberry", "strawberry", "pear", "tangerine"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["strawberry", "tangerine", "pear", "uva"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["tangerine", "uva", "pear", "violet"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["uva", "violet", "pear", "watermelon"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["violet", "watermelon", "pear", "xigua"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["pear", "watermelon", "xigua", "yuzu"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["pear", "quince", "raspberry", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["mango", "pear", "quince", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string])
    ]
    for fact in facts:
        my_engine.add_fact(fact)
    
    my_engine.operator_select(Relation("wordy", ["pear","Y","Z","W"], [DataTypes.string, DataTypes.free_var_name, DataTypes.free_var_name, DataTypes.free_var_name]),[(0,'pear',DataTypes.string)])
    table_name = '__rgxlog__wordy_select0'

    data = [
            ["pear", "kiwi", "lemon", "mango"],
            ["pear", "quince", "raspberry", "strawberry"],
            ["pear", "watermelon", "xigua", "yuzu"],
            ["pear", "quince", "raspberry", "pear"],
            ["pear", "quince", "pear", "pear"]
        ]

    expected_df = pd.DataFrame(data, columns=["col0", "col1", "col2", "col3"])
    assert expected_df.equals(my_engine.table_to_dataframe(table_name))

    table_name = '__rgxlog__wordy_select1'


    my_engine.operator_select(Relation("wordy", ["pear","Y","pear","pear"], [DataTypes.string, DataTypes.free_var_name, DataTypes.string, DataTypes.string]),[(0,'pear',DataTypes.string),(2,'pear', DataTypes.string), (3,'pear', DataTypes.string)])
    data = [
                ["pear", "quince", "pear", "pear"]
            ]

    expected_df = pd.DataFrame(data, columns=["col0", "col1", "col2", "col3"])
    assert expected_df.equals(my_engine.table_to_dataframe(table_name))

    table_name = '__rgxlog__wordy_select2'
    my_engine.operator_select(Relation("wordy", ["X","watermelon","Y","yuzu"], [DataTypes.free_var_name, DataTypes.string, DataTypes.free_var_name, DataTypes.string]),[(1,'watermelon',DataTypes.string),(3,'yuzu', DataTypes.string)])
    data = [
            ["pear", "watermelon", "xigua", "yuzu"]
        ]
    
    expected_df = pd.DataFrame(data, columns=["col0", "col1", "col2", "col3"])
    assert expected_df.equals(my_engine.table_to_dataframe(table_name))

In [ ]:
#| export
def test_operator_union() -> None:
    my_engine = SqliteEngine()

    relation1 = RelationDeclaration("relation1", [DataTypes.integer, DataTypes.integer])
    my_engine.declare_relation_table(relation1)

    facts1 = [
        AddFact("relation1", [1, 2], [DataTypes.integer, DataTypes.integer]),
        AddFact("relation1", [3, 4], [DataTypes.integer, DataTypes.integer]),
        AddFact("relation1", [5, 6], [DataTypes.integer, DataTypes.integer]),
        AddFact("relation1", [7, 8], [DataTypes.integer, DataTypes.integer]),
        AddFact("relation1", [9, 10], [DataTypes.integer, DataTypes.integer])
    ]
    for fact in facts1:
        my_engine.add_fact(fact)

    relation2 = RelationDeclaration("relation2", [DataTypes.string, DataTypes.string])
    my_engine.declare_relation_table(relation2)

    facts2 = [
        AddFact("relation2", ["apple", "banana"], [DataTypes.string, DataTypes.string]),
        AddFact("relation2", ["cherry", "date"], [DataTypes.string, DataTypes.string]),
        AddFact("relation2", ["elderberry", "fig"], [DataTypes.string, DataTypes.string]),
        AddFact("relation2", ["grape", "honeydew"], [DataTypes.string, DataTypes.string]),
        AddFact("relation2", ["kiwi", "lemon"], [DataTypes.string, DataTypes.string])
    ]
    for fact in facts2:
        my_engine.add_fact(fact)

    relation3 = RelationDeclaration("relation3", [DataTypes.integer, DataTypes.string])
    my_engine.declare_relation_table(relation3)

    facts3 = [
        AddFact("relation3", [1, "apple"], [DataTypes.integer, DataTypes.string]),
        AddFact("relation3", [2, "banana"], [DataTypes.integer, DataTypes.string]),
        AddFact("relation3", [3, "cherry"], [DataTypes.integer, DataTypes.string]),
        AddFact("relation3", [4, "date"], [DataTypes.integer, DataTypes.string]),
        AddFact("relation3", [5, "elderberry"], [DataTypes.integer, DataTypes.string])
    ]
    for fact in facts3:
        my_engine.add_fact(fact)
    
    rel1 = Relation("relation1", ["X", "Y"], [DataTypes.free_var_name, DataTypes.free_var_name])
    rel2 = Relation("relation2", ["Z", "W"], [DataTypes.free_var_name, DataTypes.free_var_name])
    rel3 = Relation("relation3", ["Z", "W"], [DataTypes.free_var_name, DataTypes.free_var_name])


    table_name = my_engine.operator_union([rel1,rel2])
    data = [
                [1, 2],
                [3, 4],
                [5, 6],
                [7, 8],
                [9, 10],
                ["apple", "banana"],
                ["cherry", "date"],
                ["elderberry", "fig"],
                ["grape", "honeydew"],
                ["kiwi", "lemon"]
            ]
    expected_df = pd.DataFrame(data, columns=["col0", "col1"])
    assert expected_df.equals(my_engine.table_to_dataframe(table_name.relation_name.split("(")[0]))    

    table_name = my_engine.operator_union([rel1,rel2, rel3])
    data = [
                [1, 2],
                [1, "apple"],
                [2, "banana"],
                [3, 4],
                [3, "cherry"],
                [4, "date"],
                [5, 6],
                [5, "elderberry"],
                [7, 8],
                [9, 10],
                ["apple", "banana"],
                ["cherry", "date"],
                ["elderberry", "fig"],
                ["grape", "honeydew"],
                ["kiwi", "lemon"]
            ]
    expected_df = pd.DataFrame(data, columns=["col0", "col1"])
    assert expected_df.equals(my_engine.table_to_dataframe(table_name.relation_name.split("(")[0]))    

In [ ]:
#| export
def test_operator_project() -> None:
    my_engine = SqliteEngine()

    relation = RelationDeclaration("wordy", [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string])

    my_engine.declare_relation_table(relation)

    facts = [
        AddFact("wordy", ["apple", "banana", "cherry", "date"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["banana", "cherry", "date", "elderberry"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["cherry", "date", "elderberry", "fig"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["date", "elderberry", "fig", "grape"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["elderberry", "fig", "grape", "honeydew"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["fig", "grape", "honeydew", "kiwi"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["grape", "honeydew", "kiwi", "lemon"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["honeydew", "kiwi", "lemon", "mango"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["pear", "kiwi", "lemon", "mango"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["kiwi", "lemon", "mango", "nectarine"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["lemon", "mango", "nectarine", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["mango", "nectarine", "pear", "quince"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["nectarine", "orange", "pear", "raspberry"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["orange", "pear", "quince", "raspberry"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["pear", "quince", "raspberry", "strawberry"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["quince", "raspberry", "strawberry", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["raspberry", "strawberry", "pear", "tangerine"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["strawberry", "tangerine", "pear", "uva"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["tangerine", "uva", "pear", "violet"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["uva", "violet", "pear", "watermelon"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["violet", "watermelon", "pear", "xigua"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["pear", "watermelon", "xigua", "yuzu"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["pear", "quince", "raspberry", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["mango", "pear", "quince", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["X", "quince", "pear", "Y"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["violet", "quince", "pear", "yuzu"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["pear", "quince", "pear", "pear"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),
        AddFact("wordy", ["X", "quince", "pear", "W"], [DataTypes.string, DataTypes.string, DataTypes.string, DataTypes.string]),

    ]
    for fact in facts:
        my_engine.add_fact(fact)
    

    project_relation = my_engine.operator_select(Relation("wordy", ["pear","Y","Z","W"], [DataTypes.string, DataTypes.free_var_name, DataTypes.free_var_name, DataTypes.free_var_name]),[(0,'pear',DataTypes.string)])
    table_name = my_engine.operator_project(project_relation, ['Y','Z','W'])

    data = [
            ["kiwi", "lemon", "mango"],
            ["quince", "raspberry", "strawberry"],
            ["watermelon", "xigua", "yuzu"],
            ["quince", "raspberry", "pear"],
            ["quince", "pear", "pear"]
        ]
    expected_df = pd.DataFrame(data, columns=["col0", "col1", "col2"])
    assert expected_df.equals(my_engine.table_to_dataframe(table_name.relation_name.split("(")[0]))

    project_relation = my_engine.operator_select(Relation("wordy", ["pear","quince","Z","W"],
                [DataTypes.string, DataTypes.string, DataTypes.free_var_name, DataTypes.free_var_name]),
                [(0,'pear',DataTypes.string),(1,'quince',DataTypes.string)])
    
    table_name = my_engine.operator_project(project_relation, ['Z','W'])

    data = [
            ["raspberry", "strawberry"],
            ["raspberry", "pear"],
            ["pear", "pear"]
        ]

    expected_df = pd.DataFrame(data, columns=["col0", "col1"])
    assert expected_df.equals(my_engine.table_to_dataframe(table_name.relation_name.split("(")[0]))

    project_relation = my_engine.operator_select(Relation("wordy", ["X","quince","pear","W"],
                [DataTypes.free_var_name, DataTypes.string, DataTypes.string, DataTypes.free_var_name]),
                [(2,'pear',DataTypes.string),(1,'quince',DataTypes.string)])
    
    table_name = my_engine.operator_project(project_relation, ['X','W'])

    data = [
            ["pear", "pear"],
            ["X", "Y"],
            ["violet", "yuzu"],
            ["X", "W"]
    ]

    expected_df = pd.DataFrame(data, columns=["col0", "col1"])
    assert expected_df.equals(my_engine.table_to_dataframe(table_name.relation_name.split("(")[0]))